In [2]:
# Block 1: Import necessary libraries
import os
import cv2
import numpy as np
import shutil
from ultralytics import YOLO
import time

# Check if libraries are properly imported
print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# Block 2: Define dataset path and check if it exists
dataset_path = r"D:\objdetection_imp_model\Image"
if os.path.exists(dataset_path):
    print(f"Dataset path found: {dataset_path}")
    # List a few sample directories to verify
    sample_dirs = os.listdir(dataset_path)[:5]  # First 5 directories
    print(f"Sample directories: {sample_dirs}")
else:
    print(f"Warning: Dataset path not found: {dataset_path}")
    # You might need to update the path

Dataset path found: D:\objdetection_imp_model\Image
Sample directories: ['A', 'B', 'C', 'D', 'E']


In [4]:
# Block 3: Define the HandGestureRecognition class
class HandGestureRecognition:
    def __init__(self, dataset_path, output_path='runs/detect'):
        self.dataset_path = dataset_path
        self.output_path = output_path
        self.model = None
        self.class_names = [chr(i + ord('A')) for i in range(26)]  # A to Z
        self.roi_x, self.roi_y = 100, 100
        self.roi_width, self.roi_height = 300, 300
        print(f"HandGestureRecognition initialized with classes: {self.class_names}")


In [5]:
# Block 4: Create an instance of the class
hgr = HandGestureRecognition(dataset_path)
print("HandGestureRecognition instance created")

HandGestureRecognition initialized with classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
HandGestureRecognition instance created


In [7]:
# Block 5: Add the prepare_dataset method to the class
def prepare_dataset(self, yaml_file="hand_gestures.yaml"):
    """
    Prepares dataset in YOLOv8 format.
    YOLOv8 requires images and labels in specific format.
    """
    # Create directory structure
    dataset_dir = "hand_gestures_dataset"
    if os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)
    
    os.makedirs(f"{dataset_dir}/train/images", exist_ok=True)
    os.makedirs(f"{dataset_dir}/train/labels", exist_ok=True)
    os.makedirs(f"{dataset_dir}/val/images", exist_ok=True)
    os.makedirs(f"{dataset_dir}/val/labels", exist_ok=True)
    
    # Create YAML file for dataset
    with open(yaml_file, "w") as f:
        f.write(f"path: {os.path.abspath(dataset_dir)}\n")
        f.write("train: train/images\n")
        f.write("val: val/images\n")
        f.write("nc: 26\n")  # Number of classes (A-Z)
        f.write(f"names: {self.class_names}\n")
    
    print("Converting dataset to YOLOv8 format...")
    
    # Process each alphabet folder
    processed_images = 0
    for idx, alphabet in enumerate(self.class_names):
        folder_path = os.path.join(self.dataset_path, alphabet)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder for {alphabet} not found at {folder_path}")
            continue
            
        images = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        print(f"Processing {alphabet}: {len(images)} images found")
        
        # Split into train (80%) and validation (20%) sets
        split_idx = int(len(images) * 0.8)
        train_images = images[:split_idx]
        val_images = images[split_idx:]
        
        # Process training images
        for img_file in train_images:
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            # Create label file (YOLOv8 format: class_id center_x center_y width height)
            dest_img_path = f"{dataset_dir}/train/images/{alphabet}_{img_file}"
            dest_label_path = f"{dataset_dir}/train/labels/{alphabet}_{os.path.splitext(img_file)[0]}.txt"
            
            # Copy image
            shutil.copy(img_path, dest_img_path)
            
            # Create label (assuming object is centered and takes up ~70% of image)
            with open(dest_label_path, "w") as f:
                f.write(f"{idx} 0.5 0.5 0.7 0.7\n")
            
            processed_images += 1
        
        # Process validation images (similar to training)
        for img_file in val_images:
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            dest_img_path = f"{dataset_dir}/val/images/{alphabet}_{img_file}"
            dest_label_path = f"{dataset_dir}/val/labels/{alphabet}_{os.path.splitext(img_file)[0]}.txt"
            
            # Copy image
            shutil.copy(img_path, dest_img_path)
            
            # Create label
            with open(dest_label_path, "w") as f:
                f.write(f"{idx} 0.5 0.5 0.7 0.7\n")
            
            processed_images += 1
    
    print(f"Dataset prepared successfully. Total processed images: {processed_images}")
    print(f"YAML file created at {yaml_file}")
    return yaml_file

# Add the method to the class
HandGestureRecognition.prepare_dataset = prepare_dataset

In [8]:
# Block 6: Prepare the dataset
yaml_file = hgr.prepare_dataset()
print(f"Dataset preparation complete. YAML file: {yaml_file}")

Converting dataset to YOLOv8 format...
Processing A: 101 images found
Processing B: 101 images found
Processing C: 101 images found
Processing D: 101 images found
Processing E: 101 images found
Processing F: 101 images found
Processing G: 101 images found
Processing H: 101 images found
Processing I: 101 images found
Processing J: 101 images found
Processing K: 101 images found
Processing L: 101 images found
Processing M: 101 images found
Processing N: 101 images found
Processing O: 101 images found
Processing P: 101 images found
Processing Q: 101 images found
Processing R: 101 images found
Processing S: 101 images found
Processing T: 101 images found
Processing U: 101 images found
Processing V: 101 images found
Processing W: 101 images found
Processing X: 101 images found
Processing Y: 101 images found
Processing Z: 101 images found
Dataset prepared successfully. Total processed images: 2626
YAML file created at hand_gestures.yaml
Dataset preparation complete. YAML file: hand_gestures.

In [11]:
# Block 7: Add the train_model method to the class
def train_model(self, yaml_file, epochs=35, img_size=640, batch_size=16):
    """Train YOLOv8 model on the prepared dataset"""
    print(f"Starting model training with {epochs} epochs...")
    
    # Initialize YOLOv8 model
    model = YOLO('yolov8n.pt')  # Start with pretrained nano model
    
    # Train the model
    results = model.train(
        data=yaml_file,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        patience=10,  # Early stopping
        name='hand_gesture_model'
    )
    
    print(f"Training completed. Model saved at {self.output_path}/hand_gesture_model")
    
    # Load the best model
    self.model = YOLO(f'{self.output_path}/hand_gesture_model/weights/best.pt')
    return self.model

# Add the method to the class
HandGestureRecognition.train_model = train_model


In [13]:
# Block 8: Train the model (you can adjust epochs for faster testing)
# Uncomment this block when you're ready to train
model = hgr.train_model(yaml_file, epochs=35)  # Reduce epochs for testing
print("Model training complete")

Starting model training with 35 epochs...


100%|██████████| 6.25M/6.25M [00:01<00:00, 5.92MB/s]


Ultralytics 8.3.91  Python-3.10.11 torch-2.6.0+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=hand_gestures.yaml, epochs=35, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=hand_gesture_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning D:\objdetection_imp_model\hand_gestures_dataset\train\labels... 2080 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2080/2080 [00:03<00:00, 534.75it/s]


train: New cache created: D:\objdetection_imp_model\hand_gestures_dataset\train\labels.cache


...

                   all        546        546      0.995      0.998      0.995      0.995

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      35/35         0G     0.1419     0.3162     0.8321         16        640: 100%|██████████| 130/130 [09:23<00:00,  4.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:48<00:00,  2.72s/it]


                   all        546        546      0.993      0.999      0.995      0.995

35 epochs completed in 6.029 hours.
Optimizer stripped from runs\detect\hand_gesture_model\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\hand_gesture_model\weights\best.pt, 6.2MB

Validating runs\detect\hand_gesture_model\weights\best.pt...
Ultralytics 8.3.91  Python-3.10.11 torch-2.6.0+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
Model summary (fused): 72 layers, 3,010,718 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:37<00:00,  2.08s/it]


                   all        546        546      0.993      0.999      0.995      0.995
                     A         21         21          1      0.965      0.995      0.995
                     B         21         21      0.993          1      0.995      0.995
                     C         21         21      0.993          1      0.995      0.995
                     D         21         21      0.993          1      0.995      0.995
                     E         21         21      0.992          1      0.995      0.995
                     F         21         21      0.993          1      0.995      0.995
                     G         21         21      0.998          1      0.995      0.995
                     H         21         21      0.993          1      0.995      0.995
                     I         21         21      0.994          1      0.995      0.995
                     J         21         21      0.993          1      0.995      0.995
                     

In [3]:
import os

# Check if model file exists
model_path = 'runs/detect/hand_gesture_model/weights/best.pt'
if os.path.exists(model_path):
    print(f"Model file found at: {model_path}")
else:
    print(f"Model file not found at: {model_path}")

Model file found at: runs/detect/hand_gesture_model/weights/best.pt


In [4]:
# Try to load the model
try:
    from ultralytics import YOLO
    model = YOLO('runs/detect/hand_gesture_model/weights/best.pt')
    print("Model loaded successfully!")
    # You can print some model information
    print(f"Model type: {type(model)}")
    print(f"Model architecture: {model.names}")
except Exception as e:
    print(f"Error loading model: {e}")
    

Model loaded successfully!
Model type: <class 'ultralytics.models.yolo.model.YOLO'>
Model architecture: {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [ ]:
import cv2
import numpy as np
import time
from datetime import datetime
import torch
from ultralytics import YOLO

class HandGestureRecognition:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.class_names = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}
    
    def real_time_detection(self):
        cap = cv2.VideoCapture(0)
        cap.set(3, 1280)  # Set width to 1280
        cap.set(4, 720)   # Set height to 720

        if not cap.isOpened():
            print("Error: Could not open webcam.")
            return
        
        print("Press 'q' to quit. Press 'Enter' to add the word to the sentence. Press 'Space' to add a space.")
        
        prev_time = 0
        gesture_history = []
        current_word = []  
        sentence = ""
        last_prediction = None  
        confidence_threshold = 0.65
        debounce_time = 0.5
        last_prediction_time = time.time()
        
        while True:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            screen_height, screen_width = frame.shape[:2]

            # ROI settings
            roi_width, roi_height = 300, 300
            roi_start_x, roi_start_y = screen_width - roi_width - 50, (screen_height - roi_height) // 2
            roi_end_x, roi_end_y = roi_start_x + roi_width, roi_start_y + roi_height
            
            roi = frame[roi_start_y:roi_end_y, roi_start_x:roi_end_x]
            results = self.model(roi, verbose=False)
            
            predicted_label, predicted_confidence = None, 0
            if results and len(results) > 0:
                for result in results:
                    boxes = result.boxes
                    if len(boxes) > 0:
                        confidences = boxes.conf
                        best_idx = np.argmax(confidences)
                        cls_id = int(boxes.cls[best_idx].item())
                        predicted_confidence = confidences[best_idx].item()
                        predicted_label = self.class_names[cls_id]

            if predicted_label and predicted_confidence >= confidence_threshold and time.time() - last_prediction_time > debounce_time:
                last_prediction_time = time.time()
                last_prediction = predicted_label
                if len(gesture_history) >= 5:
                    gesture_history.pop(0)
                gesture_history.append(predicted_label)
                if len(current_word) == 0 or (current_word and current_word[-1] != predicted_label):
                    current_word.append(predicted_label)
            
            # Overlay for ROI with stable display
            overlay = frame.copy()
            cv2.rectangle(overlay, (roi_start_x, roi_start_y), (roi_end_x, roi_end_y), (255, 200, 150), -1)
            alpha = 0.4
            cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
            cv2.rectangle(frame, (roi_start_x, roi_start_y), (roi_end_x, roi_end_y), (0, 0, 255), 2)

            # Key events for sentence formation
            key = cv2.waitKey(1) & 0xFF
            if key == ord(' '):
                if current_word:
                    sentence += ''.join(current_word) + " "
                    current_word = []
            elif key == 13:
                if current_word:
                    sentence += ''.join(current_word) + " "
                    current_word = []
            elif key == ord('q'):
                break
            
            # FPS Calculation
            curr_time = time.time()
            fps = 1 / (curr_time - prev_time) if curr_time - prev_time > 0 else 0
            prev_time = curr_time
            
            # Display info
            cv2.rectangle(frame, (20, screen_height - 120), (screen_width - 20, screen_height - 20), (0, 0, 0), -1)
            cv2.putText(frame, f'Current Word: {"".join(current_word)}', (30, screen_height - 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(frame, f'Sentence: {sentence.strip()}', (30, screen_height - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(frame, f'FPS: {int(fps)}', (roi_start_x + 10, roi_end_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            
            # Display datetime in non-overlapping area
            datetime_text = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cv2.putText(frame, datetime_text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            cv2.imshow("Hand Gesture Recognition", frame)
        
        cap.release()
        cv2.destroyAllWindows()

model_path = "runs/detect/hand_gesture_model/weights/best.pt"
hgr = HandGestureRecognition(model_path)
hgr.real_time_detection()


Press 'q' to quit. Press 'Enter' to add the word to the sentence. Press 'Space' to add a space.


In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path

# Paths
teacher_model_path = "runs/detect/hand_gesture_model/weights/best.pt"
student_model_path = "yolov8n.pt"  # Smaller model
images_dir = "D:\objdetection_imp_model\hand_gestures_dataset\train\images"  # Replace with actual path
distilled_labels_dir = "distilled_labels/"
data_yaml = "hand_gestures.yaml"  # Replace with your dataset YAML

# Load models
teacher_model = YOLO(teacher_model_path)
student_model = YOLO(student_model_path)

# Step 1: Generate pseudo-labels using teacher model
os.makedirs(distilled_labels_dir, exist_ok=True)

print("\nGenerating pseudo-labels using teacher model...")
for img_path in Path(images_dir).glob("*.jpg"):
    results = teacher_model(img_path)
    results.save_txt(save_dir=distilled_labels_dir)

print("Pseudo-labels generated and saved.")

# Step 2: Optional - Replace original labels with distilled labels
# (This step depends on your dataset structure)
# shutil.copytree(distilled_labels_dir, 'path/to/your/train/labels')

# Step 3: Train the student model using YOLO CLI (Run this manually or via subprocess)
print("\nStarting training for student model using distilled labels...")

os.system(f"yolo detect train model={student_model_path} data={data_yaml} epochs=50 imgsz=640 name=distilled_student")

print("\nStudent model training complete.")



Generating pseudo-labels using teacher model...
Pseudo-labels generated and saved.

Starting training for student model using distilled labels...
